In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [10]:
import pandas as pd
import os
import glob

# ==========================================
# 1. CONFIGURATION (Drive Edition)
# ==========================================
CONFIG = {
    # Update this to match your folder structure in Drive
    "BASE_DIR": "/content/drive/MyDrive/Lab/Lab_Data_Analysis/", 

    # We will look for these specific filenames inside the folders
    "RAW_FILENAME": "vehicle.csv",
    "AUX_FILENAME": "VEH_AUX.csv",
    "ACC_FILENAME": "accident.csv",
    
    # Codes
    "CODES": {
        "LARGE_TRUCK": [6],         # Confirmed from BODFMT
        "PASSENGER_VEH": [1, 2, 3, 4, 5], # Car, Pickup, Utility, Light Trucks
        "ROLLOVER": [1],            # Confirmed from ROLLFMT
        "WEATHER_BAD": [2, 3, 4, 10], # Rain, Sleet, Snow, Crosswinds
        
        # --- RESTORED MISSING CODES ---
        "CURVES": [2, 3, 4],        # Curve Right, Left, Unknown
        "JACKKNIF_EVENTS": [2, 3]   # First or Subsequent Event
    }
}

# ==========================================
# 2. HELPER: Auto-Detect Years
# ==========================================

def find_file_insensitive(folder, target_name):
    """
    Scans a folder for a file matching 'target_name', ignoring case.
    """
    try:
        actual_files = os.listdir(folder)
        for f in actual_files:
            if f.lower() == target_name.lower():
                return os.path.join(folder, f)
        return None
    except OSError:
        return None

def find_data_folders():
    """
    Scans the BASE_DIR and finds all folders containing the required files.
    """
    valid_years = {}
    
    # Use glob to find all 'vehicle.csv' files recursively
    search_pattern = os.path.join(CONFIG["BASE_DIR"], "**", CONFIG["RAW_FILENAME"])
    found_files = glob.glob(search_pattern, recursive=True)
    
    print(f"Scanning {CONFIG['BASE_DIR']}...")
    
    for file_path in found_files:
        folder_path = os.path.dirname(file_path)
        folder_name = os.path.basename(folder_path)
        
        if folder_name.isdigit() and len(folder_name) == 4:
            year = int(folder_name)
            valid_years[year] = folder_path
            print(f"  [+] Found data for Year: {year}")
        else:
            print(f"  [-] Found file in '{folder_name}' but it doesn't look like a year. Skipping.")
            
    return valid_years

# ==========================================
# 3. PROCESSING PIPELINE
# ==========================================
def load_and_process_year(year, folder_path):
    print(f"  [?] Checking folder: {folder_path}")
    
    # Use the smart finder for ALL files
    path_raw = find_file_insensitive(folder_path, CONFIG["RAW_FILENAME"])
    path_aux = find_file_insensitive(folder_path, CONFIG["AUX_FILENAME"])
    path_acc = find_file_insensitive(folder_path, CONFIG["ACC_FILENAME"])
    
    # Validation
    if not path_raw:
        print(f"  [!] CRITICAL: Missing '{CONFIG['RAW_FILENAME']}'")
        return None
    if not path_aux:
        print(f"  [!] CRITICAL: Missing '{CONFIG['AUX_FILENAME']}'")
        return None
    if not path_acc:
        print(f"  [!] CRITICAL: Missing '{CONFIG['ACC_FILENAME']}'")
        return None

    try:
        print(f"      Loading: {os.path.basename(path_raw)} + {os.path.basename(path_aux)} + {os.path.basename(path_acc)}")
        
        # Load Data
        df_raw = pd.read_csv(path_raw, encoding='latin1', low_memory=False)
        df_aux = pd.read_csv(path_aux, encoding='latin1', low_memory=False)
        df_acc = pd.read_csv(path_acc, encoding='latin1', low_memory=False)
        
        # Standardize Columns
        df_raw.columns = [c.upper() for c in df_raw.columns]
        df_aux.columns = [c.upper() for c in df_aux.columns]
        df_acc.columns = [c.upper() for c in df_acc.columns]
        
        rename_map = {
            'VALIGN': 'V_ALIGN',    
            'ALIGNMNT': 'V_ALIGN',  
            'WEATHER1': 'WEATHER',  
            'JACK_KNIFE': 'JACKKNIF',
            'M_JACK': 'JACKKNIF',   # The modern name (2020+)
            'J_KNIFE': 'JACKKNIF' 
        }

        df_raw.rename(columns=rename_map, inplace=True)
        df_acc.rename(columns=rename_map, inplace=True)
        
        # MERGE STEP 1: Vehicle + Aux (Inner Join)
        df_merged = pd.merge(df_raw, df_aux, on=['ST_CASE', 'VEH_NO'], how='inner', suffixes=('', '_AUX_DROP'))
        
        # MERGE STEP 2: Result + Accident (Left Join)
        df_final = pd.merge(df_merged, df_acc, on='ST_CASE', how='left', suffixes=('', '_ACC_DROP'))
        
        # Clean up
        df_final = df_final.filter(regex='^(?!.*_DROP)')

        # --- APPLY LOGIC (RESTORED MISSING CALCULATIONS) ---
        df_final['YEAR'] = year
        
        # 1. Vehicle Types
        df_final['is_large_truck'] = df_final['A_BODY'].isin(CONFIG["CODES"]["LARGE_TRUCK"])
        df_final['is_passenger_veh'] = df_final['A_BODY'].isin(CONFIG["CODES"]["PASSENGER_VEH"])
        
        # 2. Crash Dynamics
        df_final['is_curve'] = df_final['V_ALIGN'].isin(CONFIG["CODES"]["CURVES"])
        df_final['is_rollover'] = df_final['A_VROLL'].isin(CONFIG["CODES"]["ROLLOVER"])
        
        # 3. Jackknife (Handle missing column safely)
        if 'JACKKNIF' in df_final.columns:
            df_final['is_jackknife'] = df_final['JACKKNIF'].isin(CONFIG["CODES"]["JACKKNIF_EVENTS"])
        else:
            df_final['is_jackknife'] = False

        # 4. Composite
        df_final['is_truck_on_curve'] = df_final['is_large_truck'] & df_final['is_curve']
        
        return df_final

    except Exception as e:
        print(f"  [!] Error processing {year}: {e}")
        return None

# ==========================================
# 4. MAIN EXECUTION
# ==========================================
all_data = []
detected_years = find_data_folders()

if not detected_years:
    print("No valid data folders found! Check your BASE_DIR path.")
else:
    # Sort years to process in order
    for year in sorted(detected_years.keys()):
        print(f"Processing {year}...")
        df_year = load_and_process_year(year, detected_years[year])
        
        if df_year is not None:
            all_data.append(df_year)

    # Combine
    if all_data:
        final_df = pd.concat(all_data, ignore_index=True)
        print(f"\nSuccessfully imported {len(final_df)} total records from {len(all_data)} years.")
        
        # ==========================================
        # 📊 FULL STATISTICAL REPORT (No Filters)
        # ==========================================
        print("\n" + "="*50)
        print("          COMPREHENSIVE SAFETY METRICS")
        print("="*50)

        # Subsets
        trucks = final_df[final_df['is_large_truck']]
        cars = final_df[final_df['is_passenger_veh']]
        
        trucks_curve = trucks[trucks['is_curve']]
        cars_curve = cars[cars['is_curve']]
        trucks_straight = trucks[~trucks['is_curve']]

        # --- METRIC 1: CURVE SUSCEPTIBILITY ---
        # Question: Do trucks crash on curves more often than cars?
        print(f"\n1. CURVE SUSCEPTIBILITY RATIO (Frequency)")
        rate_truck_curve = (len(trucks_curve) / len(trucks) * 100) if len(trucks) > 0 else 0
        rate_car_curve = (len(cars_curve) / len(cars) * 100) if len(cars) > 0 else 0
        
        print(f"   - % of Car Crashes on Curves:   {rate_car_curve:.2f}%")
        print(f"   - % of Truck Crashes on Curves: {rate_truck_curve:.2f}%")
        
        multiplier = rate_truck_curve / rate_car_curve if rate_car_curve > 0 else 0
        print(f"   -> Multiplier (Truck/Car):      {multiplier:.2f}x")

        # --- METRIC 2: ROLLOVER PROBABILITY ---
        # Question: If a truck is already on a curve, how likely is it to roll over?
        print(f"\n2. ROLLOVER PROBABILITY (On Curves)")
        
        roll_rate_truck = (trucks_curve['is_rollover'].mean() * 100) if len(trucks_curve) > 0 else 0
        roll_rate_car = (cars_curve['is_rollover'].mean() * 100) if len(cars_curve) > 0 else 0
        
        print(f"   - Car Rollover Rate:   {roll_rate_car:.2f}%")
        print(f"   - Truck Rollover Rate: {roll_rate_truck:.2f}%")
        
        multiplier = roll_rate_truck / roll_rate_car if roll_rate_car > 0 else 0
        print(f"   -> Multiplier (Truck/Car):      {multiplier:.2f}x")

        # --- METRIC 3: JACKKNIFE INDEX ---
        # Question: Does the curve make the truck unstable (Jackknife)?
        print(f"\n3. JACKKNIFE INDEX (Trucks Only)")
        
        jk_rate_curve = (trucks_curve['is_jackknife'].mean() * 100) if len(trucks_curve) > 0 else 0
        jk_rate_straight = (trucks_straight['is_jackknife'].mean() * 100) if len(trucks_straight) > 0 else 0
        
        print(f"   - Jackknife Rate (Straight): {jk_rate_straight:.2f}%")
        print(f"   - Jackknife Rate (Curve):    {jk_rate_curve:.2f}%")
        
        multiplier = jk_rate_curve / jk_rate_straight if jk_rate_straight > 0 else 0
        print(f"   -> Multiplier (Curve/Straight): {multiplier:.2f}x")

        # --- METRIC 4: LETHALITY INDEX ---
        # Question: When they do crash on a curve, how many people die?
        print(f"\n4. LETHALITY INDEX (Fatalities per Crash on Curves)")
        
        avg_fatal_truck = trucks_curve['FATALS'].mean() if len(trucks_curve) > 0 else 0
        avg_fatal_car = cars_curve['FATALS'].mean() if len(cars_curve) > 0 else 0
        
        print(f"   - Avg Fatalities (Car Crash):   {avg_fatal_car:.3f}")
        print(f"   - Avg Fatalities (Truck Crash): {avg_fatal_truck:.3f}")
        
        multiplier = avg_fatal_truck / avg_fatal_car if avg_fatal_car > 0 else 0
        print(f"   -> Multiplier (Truck/Car):      {multiplier:.2f}x")
        
        print("\n" + "="*50)

    else:
        print("No data loaded.")
    

Scanning /content/drive/MyDrive/Lab/Lab_Data_Analysis/...
  [+] Found data for Year: 2023
  [+] Found data for Year: 2022
  [+] Found data for Year: 2021
  [+] Found data for Year: 2020
Processing 2020...
  [?] Checking folder: /content/drive/MyDrive/Lab/Lab_Data_Analysis/2020
      Loading: vehicle.csv + VEH_AUX.CSV + accident.csv
Processing 2021...
  [?] Checking folder: /content/drive/MyDrive/Lab/Lab_Data_Analysis/2021
      Loading: vehicle.csv + VEH_AUX.CSV + accident.csv
Processing 2022...
  [?] Checking folder: /content/drive/MyDrive/Lab/Lab_Data_Analysis/2022
      Loading: vehicle.csv + veh_aux.csv + accident.csv
Processing 2023...
  [?] Checking folder: /content/drive/MyDrive/Lab/Lab_Data_Analysis/2023
      Loading: vehicle.csv + veh_aux.csv + accident.csv

Successfully imported 235438 total records from 4 years.

          COMPREHENSIVE SAFETY METRICS

1. CURVE SUSCEPTIBILITY RATIO (Frequency)
   - % of Car Crashes on Curves:   18.56%
   - % of Truck Crashes on Curves: 15.0